In [1]:
# Trick to import local code:
import sys
sys.path = ['/home/mp/michelj/src/sensorsio/src'] + sys.path

In [2]:
import rasterio as rio
import numpy as np
from matplotlib import pyplot
from sensorsio import sentinel2, utils

# Creating a dataset

Constructor of ```sentinel2.Sentinel2``` class reads the path to the ***Sentinel2 L2A*** product:

In [3]:
dataset = sentinel2.Sentinel2('/datalake/S2-L2A-THEIA/31TDH/2019/05/31/SENTINEL2B_20190531-105916-927_L2A_T31TDH_C_V2-2/')

Printing the returned objects gives basic information on the product:

In [4]:
print(dataset)

SENTINEL2B, 2019-05-31 00:00:00, 31TDH


Among the object attributes, one can find information on acquisition date as a ```datetime.datetime``` object, acquisition year, and day of year:

In [5]:
print(f"date: {dataset.date}, year: {dataset.year}, day of year: {dataset.day_of_year}")

date: 2019-05-31 00:00:00, year: 2019, day of year: 151


One can also read the satellite id, the MGRS tile id and the corresponding coordinate reference system:

In [6]:
print(f"Satellite: {dataset.satellite.value}, MGRS tile: dataset.tile, CRS: {dataset.crs}")

Satellite: SENTINEL2B, MGRS tile: dataset.tile, CRS: EPSG:32631


Last, the object also contains the geographic bounds expressed in the given CRS.

In [7]:
print(f"Product bounds: {dataset.bounds}")

Product bounds: BoundingBox(left=399960.0, bottom=4690200.0, right=509760.0, top=4800000.0)


# Basic reading

## Reading to numpy

### Selecting bands

```sentinel2.Sentinel2.GROUP_10M``` is a convenient variable to select all 10m bands:

In [8]:
sentinel2.Sentinel2.GROUP_10M

[<Band.B2: 'B2'>, <Band.B3: 'B3'>, <Band.B4: 'B4'>, <Band.B8: 'B8'>]

In a similar way, ```sentinel2.Sentinel2.GROUP_20M``` and ```sentinel2.Sentinel2.GROUP_60M``` exist:

In [9]:
sentinel2.Sentinel2.GROUP_20M

[<Band.B5: 'B5'>,
 <Band.B6: 'B6'>,
 <Band.B7: 'B7'>,
 <Band.B8A: 'B8A'>,
 <Band.B11: 'B11'>,
 <Band.B12: 'B12'>]

In [ ]:
sentinel2.Sentinel2.GROUP_60M

One can of course make her own selection of bands:

In [ ]:
my_bands = [sentinel2.Sentinel2.B2, sentinel2.Sentinel2.B8A]

### A first read

In the following we will use the ```sentinel2.Sentinel2.GROUP_10M``` to read all 10m bands to a numpy array:

In [ ]:
bands, masks, xcoords, ycoords, crs = dataset.read_as_numpy(sentinel2.Sentinel2.GROUP_10M)

The first returned value is a ```numpy.ndarray``` with shape ```[bands, height, width]``` containing pixels reflectances:

In [ ]:
bands.shape

The second returned value is a ```numpy.ndarray``` with shape ```[masks, height, width]``` containing masks values (by default, the 4 masks are read):

In [ ]:
masks.shape

3rd and 4th returned values are the row and col coordinates expressed in the given crs:

In [ ]:
xcoords, ycoords

Last returned value is the crs as a string:

In [ ]:
crs

By default, pixels values are scaled to $[0,1]$, and missing values are set to ```numpy.nan``` (hence the use of ```nanmin``` and ```nanmax``` in the following):

In [ ]:
np.nanmin(bands, axis=(1,2)), np.nanmax(bands, axis=(1,2))

### Displaying the image

The ```utils``` module provides a convenient function that will prepare the array for display with ```matplotlib```. The ```bands ``` argument indicates which slice of the first dimension to map to the red, green and blue channels. ```dmin``` and ```dmax``` arguments allows to select the displayed range for each band. 

In [ ]:
arr_rgb, dmin, dmax = utils.rgb_render(bands, bands=[2,1,0], 
                                 dmin=np.array([0., 0., 0.]), 
                                 dmax=np.array([0.2,0.2,0.2]))
pyplot.imshow(arr_rgb)

## Reading to xarray Dataset

The ```xarray``` library provides a nice decoration of ```numpy``` arrays for even more abstraction:

In [ ]:
xrds = dataset.read_as_xarray(sentinel2.Sentinel2.GROUP_10M)

In [ ]:
print(xrds)

The created datasets contains several variables, with the requested bands and masks:

In [ ]:
xrds.data_vars

Those variables are indexed with 3 sets of coordinates: x coordinates, y coordinates and time coordinates:

In [ ]:
xrds.coords

The time coordinate has only a single value (since a dataset represents one acquisition date) but it would be very easy to stack several datasets together along the time dimension.

It is rather easy to use ```xarray``` to derive new variables:

In [ ]:
ndvi = (xrds.B8-xrds.B4)/(1e-6 + xrds.B8 + xrds.B4)
print(ndvi)

Of course, one can always access the underlying ```numpy``` arrays:

In [ ]:
ndvi.values

# Advanced read

In this section, examples will be demonstrated with ```read_as_xarray(...)``` but one can achieve similar operations with ```read_as_numpy(...)```.

## Reading a spatial subset

The ```region``` argument allows to read a spatial subset of the data by providing ```(startx, starty, endx, endy)``` integer pixel coordinates tuple. Beware that those coordinates relate to the coordinates in the array after resampling or reprojection, and not input image coordinates.

In [ ]:
print(dataset.read_as_xarray(sentinel2.Sentinel2.GROUP_10M, 
                             region=(1000,1000,1200,1200)))

The ```region``` argument also accepts a ```rio.coords.BoundingBox``` object representing a geo-referenced bounding-box in the given ```crs``` (default to input image ```crs```):

In [ ]:
georoi = rio.coords.BoundingBox(399960.0, 4690200.0, 401960.0, 4692200.0) # left, bottom, right, top

In [ ]:
print(dataset.read_as_xarray(sentinel2.Sentinel2.GROUP_10M,
                             region=georoi))

## Read bands with different resolutions, and changing resolution

All bands are resampled to the target resolution set by the ```resolution``` argument, which defaults to 10m. The interpolation algorithm can be set using the ```algorithm``` keyword and selected among the algorithm available in gdal/rasterio: https://rasterio.readthedocs.io/en/latest/api/rasterio.enums.html#rasterio.enums.Resampling (default is ```rio.enums.Resampling.cubic```).

In [ ]:
print(dataset.read_as_xarray([sentinel2.Sentinel2.B2, sentinel2.Sentinel2.B6, sentinel2.Sentinel2.B11], # A 10m, a 20m and a 60m bands 
                       resolution=10, # The target resolution
                       region=georoi,
                       algorithm=rio.enums.Resampling.bilinear))

One can of course set a lower target resolution and change the interpolation algorithm accordingly:

In [ ]:
print(dataset.read_as_xarray([sentinel2.Sentinel2.B2, sentinel2.Sentinel2.B6, sentinel2.Sentinel2.B11], # A 10m, a 20m and a 60m bands 
                             resolution=60, # The target resolution
                             region=georoi,
                             algorithm=rio.enums.Resampling.average))

## Changing projection and sampling grid

The ```crs``` argument allows to resample the data into a different Coordinates Reference System  upon reading:

In [ ]:
print(dataset.read_as_xarray(sentinel2.Sentinel2.GROUP_10M, crs='epsg:2154')) # Lambert93 EPSG code

One can also change the image sampling grid by overloading the geographic bounds:

In [ ]:
newbounds = rio.coords.BoundingBox(399965.0, 4690205.0, 401965.0, 4692205.0) # left, bottom, right, top

In [ ]:
print(dataset.read_as_xarray(sentinel2.Sentinel2.GROUP_10M, bounds=newbounds))

***Note:*** This is different from using the region keyword which only does a subsetting. The ```bounds``` argument allows to modify the image sampling grid to match the grid of another image for instance.

## Using StackReg offsets

```StackReg``` provides offsets to apply to the image origin in order to getter better multi-temporal registration. Those offsets can be passed to the ```Sentinel2``` dataset constructor:

In [ ]:
dataset = sentinel2.Sentinel2('/datalake/S2-L2A-THEIA/31TDH/2019/05/31/SENTINEL2B_20190531-105916-927_L2A_T31TDH_C_V2-2/', offsets=[-1.5, 2.5]) # x and y offsets in meters